In [1]:
import numpy as np
import pandas as pd
from datetime import datetime
from dateutil.relativedelta import relativedelta
from scipy import optimize

# Change the representation of float format to show the values as they are (Without wrong roundings).
pd.set_option('display.float_format', str)

class bond_pricer:
    
    def __init__(self, 
                 principal_value, 
                 coupon_rate,
                 settlement_date, 
                 maturity_date,
                 coupon_frequency=1,
                 convention='30/360',
                 yield_to_maturity=None, 
                 dirty_price=None,
                 clean_price=None):
        
        self.principal_value = principal_value
        self.coupon_rate = coupon_rate
        self.coupon_frequency = coupon_frequency
        self.coupon_amount = self.principal_value * (self.coupon_rate / self.coupon_frequency)
        self.settlement_date = datetime.strptime(settlement_date, '%d/%m/%Y').date()
        self.maturity_date = datetime.strptime(maturity_date, '%d/%m/%Y').date()
        self.convention = convention
        self.coupon_months = int(12 / self.coupon_frequency)  
        
        # Compute cash flow dates
        cflow_date = self.maturity_date
        cflow_dates = [self.maturity_date]
        while cflow_date + relativedelta(months=-self.coupon_months) > self.settlement_date:
            cflow_date += relativedelta(months=-self.coupon_months)
            cflow_dates.append(cflow_date)        
        self.cash_flow_dates = cflow_dates[::-1]
        
        # Compute accrued interest
        self.next_coupon_date = self.cash_flow_dates[0]
        self.previous_coupon_date = self.next_coupon_date + relativedelta(months=-self.coupon_months)
        if self.convention == 'ACT/ACT':
            self.accrued_days = (self.settlement_date - self.previous_coupon_date).days    
            self.curr_coupon_days = (self.next_coupon_date - self.previous_coupon_date).days
            self.accrual_period = self.accrued_days / self.curr_coupon_days
            self.accrued_interest = self.principal_value * (self.coupon_rate / self.coupon_frequency) * self.accrual_period
        elif self.convention == '30/360':
            d1 = min(30, self.previous_coupon_date.day)
            if d1 == 30:
                d2 = min(d1, self.settlement_date.day)
            else:
                d2 = self.settlement_date.day
            self.accrued_days = 360 * (self.settlement_date.year - self.previous_coupon_date.year) + 30 * (self.settlement_date.month - self.previous_coupon_date.month) + d2 - d1
            self.curr_coupon_days = 360 / self.coupon_frequency
            self.accrual_period = self.accrued_days / self.curr_coupon_days
            self.accrued_interest = self.principal_value * (self.coupon_rate / self.coupon_frequency) * self.accrual_period
        elif self.convention == 'ACT/360':
            self.accrued_days = (self.settlement_date - self.previous_coupon_date).days    
            self.curr_coupon_days = 360 / self.coupon_frequency
            self.accrual_period = self.accrued_days / self.curr_coupon_days
            self.accrued_interest = self.principal_value * (self.coupon_rate / self.coupon_frequency) * self.accrual_period
        elif self.convention == 'ACT/365':
            self.accrued_days = (self.settlement_date - self.previous_coupon_date).days
            self.curr_coupon_days = (self.next_coupon_date - self.previous_coupon_date).days
            self.accrual_period = self.accrued_days / 365
            self.accrued_interest = self.principal_value * (self.coupon_rate / self.coupon_frequency) * self.accrual_period
        
        # Compute cash flow amounts and time to cash flows and maturity
        first_tau = (self.curr_coupon_days - self.accrued_days) / self.curr_coupon_days        
        cash_flow_amounts = []
        taus = []
        for i in range(len(self.cash_flow_dates)):
            cash_flow_amounts.append(self.coupon_amount)
            taus.append((first_tau + i) / self.coupon_frequency)        
        cash_flow_amounts[-1] += self.principal_value
        self.cash_flows = np.array(cash_flow_amounts)
        self.taus = np.array(taus)
        self.time_to_maturity = self.taus[-1]
        
        # Compute clean and dirty prices
        if dirty_price:
            self.dirty_price = dirty_price
            self.clean_price = self.dirty_price - self.accrued_interest
        elif clean_price:
            self.clean_price = clean_price
            self.dirty_price = self.clean_price + self.accrued_interest
        elif yield_to_maturity:
            self.yield_to_maturity = yield_to_maturity
            self.ytm_adjusted = self.coupon_frequency*np.log(1 + self.yield_to_maturity / self.coupon_frequency)
            self.discount_factors = np.exp(-self.ytm_adjusted * self.taus)
            self.dirty_price = np.sum(self.cash_flows * self.discount_factors)
            self.clean_price = self.dirty_price - self.accrued_interest
        
        # Compute current yield
        self.current_yield = (self.coupon_rate * self.principal_value) / self.dirty_price
        
        # Compute yield to maturity
        if yield_to_maturity:
            self.yield_to_maturity = yield_to_maturity
            self.ytm_adjusted = self.coupon_frequency * np.log(1 + self.yield_to_maturity / self.coupon_frequency)
        else:
            def get_bond_price(yield_to_maturity, cash_flows, taus):
                return np.sum(cash_flows * np.exp(-yield_to_maturity * taus))
            
            get_ytm = lambda ytm: get_bond_price(ytm, self.cash_flows, self.taus) - self.dirty_price
            
            self.ytm_adjusted = optimize.newton(get_ytm, 0.04)
            self.yield_to_maturity = (np.exp(self.ytm_adjusted / self.coupon_frequency) - 1) * self.coupon_frequency
            self.discount_factors = np.exp(-self.ytm_adjusted * self.taus)
       
        # Compute duration
        self.first_derivative = np.sum(-self.taus * self.cash_flows * self.discount_factors)
        self.duration_macaulay = -1/self.dirty_price * self.first_derivative
        self.duration_modified = self.duration_macaulay / (1 + self.yield_to_maturity / self.coupon_frequency)                                                        
        
        # Compute convexity
        self.second_derivative = np.sum(self.taus**2 * self.cash_flows * self.discount_factors)
        self.convexity = 1/self.dirty_price * self.second_derivative
        
        # Generate summary table
        index = ['Dirty price','Clean price', 'Accrued interest', 'Face value', 'Coupon rate', 'Coupon frequency', 'Yield to maturity', 
                 'Time to maturity', 'Macaulay duration', 'Modified duration', 'Convexity', 'Settlement date', 'Maturity date', 'Convention']
        data = list(np.round([self.dirty_price, self.clean_price, self.accrued_interest], 7))
        data += list(np.round([self.principal_value, self.coupon_rate, self.coupon_frequency, 
                self.yield_to_maturity, self.time_to_maturity, self.duration_macaulay, self.duration_modified, self.convexity], 4))
        data += [self.settlement_date, self.maturity_date, self.convention]
        self.summary = pd.DataFrame(data, index, columns=[''])
        
    # Approximate new price after delta_ytm change in yield to maturity
    def approx_new_price(self, delta_ytm):
        return self.dirty_price * (1 + (-self.duration_modified * delta_ytm + 0.5 * self.convexity * delta_ytm**2))
    
    # Summary of the bond metrics
    def print_summary(self):
        print(self.summary)

In [2]:
def get_bond_parameters(bond):
    """ Get the necessary parameters for calculations of one single bond 
    
    Parameters:
        bond (pd.Series): Pandas Series object which represents data of the bond.
    
    Returns:
        principal_value (int)
        coupon_rate (float): coupon rate divided by 100 as it is in percents
        coupon_frequency (int)
        settlement_date (str): '%d/%m/%Y'
        maturity_date (str): '%d/%m/%Y'
        yield_to_maturity (float): yield to maturity divided by 100 as it is in percents
        convention (str)
    """
    
    principal_value = bond['Principal Value']
    
    # Coupon rate (%)
    coupon_rate = bond['Coupon Rate (%)'] / 100
    
    coupon_frequency = bond['Coupon Frequency']
    
    # Change date representation from %Y-%m-%d --> %d/%m/%Y
    settlement_date = '/'.join(bond['Settlement Date'].split('-')[::-1]) 
    maturity_date = '/'.join(bond['Maturity Date'].split('-')[::-1])
    
    # Yield to maturity (%)
    yield_to_maturity = bond['Yield to Maturity (%)'] / 100
    
    convention = bond['Convention']
    
    return principal_value, coupon_rate, coupon_frequency, settlement_date, maturity_date, yield_to_maturity, convention


In [3]:
data = pd.read_csv('cleaned_data')
data

,ISIN,Coupon Rate (%),Maturity Date,Dirty Price,Yield to Maturity (%),Coupon Frequency,Principal Value,Settlement Date,Convention
0,AMGB1029A235,10.0,2023-10-29,99.4308863,11.3292,2,100,2023-05-02,ACT/ACT
1,AMGB1029A250,11.0,2025-10-29,99.0564787,11.4876,2,100,2023-05-02,ACT/ACT
2,AMGB2029A366,13.0,2036-10-29,110.5500513,11.4605,2,100,2023-05-02,ACT/ACT
3,AMGB2029A374,12.5,2037-10-29,107.3681638,11.4602,2,100,2023-05-02,ACT/ACT
4,AMGB1029A276,10.0,2027-10-29,95.02643,11.4709,2,100,2023-05-02,ACT/ACT
5,AMGB20172327,13.0,2032-02-17,110.9941415,11.4626,2,100,2023-05-02,ACT/ACT
6,AMGB20072287,11.0,2028-02-07,100.8239823,11.4701,2,100,2023-05-02,ACT/ACT
7,AMGB30163472,13.0,2047-03-16,114.1387587,11.4584,2,100,2023-05-02,ACT/ACT
8,AMGN36294269,9.0,2026-04-29,93.9323582,11.4825,2,100,2023-05-02,ACT/ACT
9,AMGN60294284,9.25,2028-04-29,91.8127554,11.4694,2,100,2023-05-02,ACT/ACT


In [4]:
def show_dirty_prices(bonds):
    """ Shows the dirty prices of our bonds with calculated dirty prices
    
    Parameters:
        bonds (pd.DataFrame): pd.DataFrame object which represents our bonds data.
        
    Returns:
        dirty_prices_comparison (pd.DataFrame): pd.DataFrame object with 2 columns 'Real Dirty Price', 'Calculated Dirty Price'
    """
    
    calculated_dirty_prices = []
    
    for i in range(len(bonds)):
        bond_i = bonds.iloc[i]
        principal_value, coupon_rate, coupon_frequency, settlement_date, maturity_date, yield_to_maturity, convention = get_bond_parameters(bond_i)
        bond = bond_pricer(principal_value=principal_value, coupon_rate=coupon_rate, coupon_frequency=coupon_frequency, 
                    settlement_date=settlement_date, maturity_date=maturity_date, 
                    yield_to_maturity=yield_to_maturity, convention=convention)
        calculated_dirty_prices.append(np.round(bond.dirty_price, 7))
    
    dirty_prices_comparison = pd.DataFrame({'Real Dirty Price': bonds['Dirty Price'], 'Calculated Dirty Price': calculated_dirty_prices})
        
    return dirty_prices_comparison
    

In [5]:
show_dirty_prices(data)

,Real Dirty Price,Calculated Dirty Price
0,99.4308863,99.4608286
1,99.0564787,99.0564787
2,110.5500513,110.5500513
3,107.3681638,107.3681638
4,95.02643,95.02643
5,110.9941415,110.9941415
6,100.8239823,100.8239823
7,114.1387587,114.1387587
8,93.9323582,93.9323582
9,91.8127554,91.8127554
